In [1]:
import pandas as pd
import psycopg2
import datetime
import matplotlib.pyplot as plt
%matplotlib inline  
import seaborn as sns

## What is the purpose of this script:

I want to know:
* which datasets recently published (2018) have a lot of incomplete records (here I set the threshold to 100% incomplete records),
* and what is the most frequent reason for inocmpleteness.

Then I want to aggregate this information per endorsing node and their continent.


## First, get the information from hive

The definition on incompleteness can be found on any [country page](https://www.gbif.org/country/SE/publishing), it is the following:
>A record is here defined to be complete if it includes an identification at least to species rank, valid coordinates, a full date of occurrence and a given basis of record (e.g. Observation, specimen etc).

This is the query I ran, it is based on the [following ones](https://github.com/gbif/analytics/blob/master/hive/process/occ_complete_v2.q) (which are used to calculate the number of complete records accross snapshots).

I decided not to use the snapshots.

This query gives us mor infomration than needed but I thought it would be usedful to have, in case we get asked more details about the record completeness.

Unlike for the [queries this was inspired from](https://github.com/gbif/analytics/blob/master/hive/process/occ_complete_v2.q), this is rgouped by datasets instead of by snapshots.

``` sql
SELECT
datasetkey,
basisofrecord,
COALESCE(
    CASE WHEN species IS NOT NULL AND species != taxonid THEN 'Infraspecies' ELSE NULL END,
    CASE WHEN species IS NOT NULL THEN 'Species' ELSE NULL END,
    CASE WHEN genus IS NOT NULL THEN 'Genus' ELSE NULL END,
    CASE WHEN family IS NOT NULL THEN 'Family' ELSE NULL END,
    CASE WHEN order_ IS NOT NULL THEN 'Order' ELSE NULL END,
    CASE WHEN class IS NOT NULL THEN 'Class' ELSE NULL END,
    CASE WHEN phylum IS NOT NULL THEN 'Phylum' ELSE NULL END,
    CASE WHEN kingdom IS NOT NULL AND kingdom != 0 THEN 'Kingdom' ELSE NULL END,
    "Unknown"
  ) AS rank,
  COALESCE(
    CASE WHEN hascoordinate = TRUE THEN 'Georeferenced' ELSE NULL END,
    CASE WHEN countrycode IS NOT NULL THEN 'CountryOnly' ELSE NULL END,
    "Unknown"
  ) AS geospatial,
  COALESCE(
    CASE WHEN year IS NOT NULL AND month IS NOT NULL AND day IS NOT NULL THEN 'YearMonthDay' ELSE NULL END,
    CASE WHEN year IS NOT NULL AND month IS NOT NULL THEN 'YearMonth' ELSE NULL END,
    CASE WHEN year IS NOT NULL THEN 'YearOnly' ELSE NULL END,
    "Unknown"
  ) AS temporal,
  COUNT(*) AS count
FROM occurrence_hdfs
WHERE to_date(from_unixtime(cast(fragmentcreated/1000 as INT))) >= "2018-01-01" AND to_date(from_unixtime(cast(fragmentcreated/1000 as INT))) < "2019-01-01"
GROUP BY
  datasetkey,
  basisofrecord,
  COALESCE(
    CASE WHEN species IS NOT NULL AND species != taxonid THEN 'Infraspecies' ELSE NULL END,
    CASE WHEN species IS NOT NULL THEN 'Species' ELSE NULL END,
    CASE WHEN genus IS NOT NULL THEN 'Genus' ELSE NULL END,
    CASE WHEN family IS NOT NULL THEN 'Family' ELSE NULL END,
    CASE WHEN order_ IS NOT NULL THEN 'Order' ELSE NULL END,
    CASE WHEN class IS NOT NULL THEN 'Class' ELSE NULL END,
    CASE WHEN phylum IS NOT NULL THEN 'Phylum' ELSE NULL END,
    CASE WHEN kingdom IS NOT NULL AND kingdom != 0 THEN 'Kingdom' ELSE NULL END,
    "Unknown"
  ),
  COALESCE(
    CASE WHEN hascoordinate = TRUE THEN 'Georeferenced' ELSE NULL END,
    CASE WHEN countrycode IS NOT NULL THEN 'CountryOnly' ELSE NULL END,
    "Unknown"
  ),
  COALESCE(
    CASE WHEN year IS NOT NULL AND month IS NOT NULL AND day IS NOT NULL THEN 'YearMonthDay' ELSE NULL END,
    CASE WHEN year IS NOT NULL AND month IS NOT NULL THEN 'YearMonth' ELSE NULL END,
    CASE WHEN year IS NOT NULL THEN 'YearOnly' ELSE NULL END,
    "Unknown"
  );```
  
## Second, get information from the registry

This is to be able to summarise the completness information per year of dataset created and per node/continent.

In [2]:
conn = psycopg2.connect(dbname='prod_b_registry', 
                        user='', 
                        host='pg1.gbif.org', 
                        password='')
cur = conn.cursor()
# This is the query for the registry
query = """SELECT d.key, d.created, node.title, node.continent, node.country
           FROM dataset d JOIN organization o ON d.publishing_organization_key = o.key
           JOIN node ON o.endorsing_node_key = node.key;"""
# Queries the GBIF registry
cur.execute(query)
res_query = cur.fetchall()

dataset_per_publishing_country = pd.DataFrame(res_query, columns=['UUID',
                                                                  'created',
                                                                  'nodeTitle',
                                                                  'nodeContinent',
                                                                  'nodeCountry'])
dataset_per_publishing_country = dataset_per_publishing_country.set_index("UUID")
cur.close()
conn.close()

## Load information from saved hive table and process it

In [3]:
dq_2018_path = "../2018_datasets_query20190117.csv"
dq_2017_path = "../2017_datasets_query20190117.csv"
dq_2018 = pd.read_table(dq_2018_path, sep = ",")
dq_2017 = pd.read_table(dq_2017_path, sep = ",")

In [4]:
def complete_name(dq):
    return ((dq["rank"] == "Species") | (dq["rank"] == "Infraspecies"))

def complete_basisofrecord(dq):
    return dq["basisofrecord"] != "UNKNOWN"
    
def complete_geospatial(dq):
    return dq["geospatial"] == "Georeferenced"

def complete_temporal(dq):
    return dq["temporal"] == "YearMonthDay"

def complete_records(dq_year):
    '''
    return a mask for a given dataset of complete records
    '''
    complete = complete_name(dq_year) & complete_basisofrecord(dq_year) & complete_geospatial(dq_year) & complete_temporal(dq_year)
    return complete

In [5]:
def number_of_complete_records_for_year(dq_year):
    '''
    Return number of complete and total records published in a given year
    in addition to the main issues for incomplete records
    '''
    records_stats = pd.DataFrame(columns=["complete", "total"],
                                 index=dq_year.groupby(["datasetkey"]).first().index)
    complete = complete_records(dq_year)
    records_stats["complete"] = dq_year[complete].groupby(["datasetkey"])["count"].sum()
    records_stats["total"] = dq_year.groupby(["datasetkey"])["count"].sum()
    records_stats = records_stats.fillna(0)
    
    # Check which is the most frequent incompleteness reason
    reason_incomplete = dq_year[~complete].sort_values(["datasetkey", "count"],
                                                       ascending=False).groupby("datasetkey").first()
    records_stats = pd.concat([records_stats, reason_incomplete], axis = 1)
    return records_stats

In [6]:
def select_ds_by_year_registry(dataset_per_publishing_country, year):
    '''
    return a mask to select datasets created a certain year
    '''
    before_that_year = pd.to_datetime(dataset_per_publishing_country.created, utc=True) >= datetime.datetime(year=year, month=1, day=1)
    after_that_year = pd.to_datetime(dataset_per_publishing_country.created, utc=True) < datetime.datetime(year=year+1, month=1, day=1)
    return (before_that_year & after_that_year)

In [7]:
def completeness_for_ds_published_in_year(dq_year, dataset_per_publishing_country, year):
    '''
    returns datasets filtered for a given year and merged
    '''
    dq_stats = number_of_complete_records_for_year(dq_year)
    mask = select_ds_by_year_registry(dataset_per_publishing_country, year)
    dq_ds_stats = pd.concat([dataset_per_publishing_country[mask], dq_stats], axis = 1, join = 'inner')

    return dq_ds_stats

In [8]:
def main_incompleteness_reason(dataset):
    '''
    For a given subset, return the most frequent reason
    for data incompleteness
    '''
    res = pd.DataFrame(columns=["freq"])
    res.at["basis of record", "freq"] = dataset[~complete_basisofrecord(dataset)].shape[0]
    res.at["taxon rank", "freq"] = dataset[~complete_name(dataset)].shape[0]
    res.at["geospatial", "freq"] = dataset[~complete_geospatial(dataset)].shape[0]
    res.at["temporal", "freq"] = dataset[~complete_temporal(dataset)].shape[0]
    return res.idxmax()[0]

In [9]:
def groupby_stats(dq_year, dataset_per_publishing_country, groupby, threshold, year):
    '''
    Aggregates statistics per node or continent
    '''
    # Column titles
    percentage_below_thresh = "Pct of "+str(year)+" datasets with incomplete data"
    incompleteness = "Main reason of incompleteness in "+str(year)
    
    # Calculate the number of complete records per dataset
    datasets_stats = completeness_for_ds_published_in_year(dq_year, dataset_per_publishing_country, year)
    datasets_stats["complete_datasets_mask"] = datasets_stats["complete"]/datasets_stats["total"] <= threshold
    
    # Aggregate statistics
    stats_per_groupby = pd.DataFrame(columns=[percentage_below_thresh])
    # Calculate percentage of datasets with incomplete data records
    stats_per_groupby[percentage_below_thresh] = datasets_stats.groupby(groupby)["complete_datasets_mask"].sum()*100/datasets_stats.groupby([groupby])["complete_datasets_mask"].count()
    # Get most frequent reason for incompleteness
    for subgroup in stats_per_groupby.index.tolist():
        subset = datasets_stats[datasets_stats[groupby] == subgroup]
        stats_per_groupby.at[subgroup, incompleteness] = main_incompleteness_reason(subset)
    return stats_per_groupby

In [10]:
stats_per_continent_2018 = groupby_stats(dq_2018, dataset_per_publishing_country, "nodeContinent", 0, 2018)
stats_per_continent_2017 = groupby_stats(dq_2018, dataset_per_publishing_country, "nodeContinent", 0, 2017)
stats_per_node_2018 = groupby_stats(dq_2018, dataset_per_publishing_country, "nodeTitle", 0, 2018)
stats_per_node_2017 = groupby_stats(dq_2018, dataset_per_publishing_country, "nodeTitle", 0, 2017)

In [11]:
stats_per_continent = pd.concat([stats_per_continent_2018, stats_per_continent_2017], axis=1)
stats_per_node = pd.concat([stats_per_node_2018, stats_per_node_2017], axis=1)

In [12]:
writer = pd.ExcelWriter('data_quality_for_2018.xlsx')
stats_per_continent.to_excel(writer,'CONTINENT')
stats_per_node.to_excel(writer,'ENDOSING NODE')
writer.save()